In [ ]:
import os
import json
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle
import sys
sys.path.insert(0,'..')
from xbrl.algs.nnmodel import MLLinearNetwork
from xbrl.envs import hlsutils

%matplotlib inline

def flatten_dict(d):
    out = {}
    for key, val in d.items():
        if isinstance(val, dict):
            val = [val]
        if isinstance(val, list):
            for subdict in val:
                deeper = flatten_dict(subdict).items()
                out.update({key + '.' + key2: val2 for key2, val2 in deeper})
        else:
            out[key] = val
    return out

# EPS_GREEDY
prefix = "/private/home/pirotta/lrcb/wheel_0524/"
folders = [
    # f"{prefix}nnegreedy_mse",
    # f"{prefix}nnegreedy_minfeat",
    f"{prefix}nnegreedy_rayleigh",
]

df = None
result_keys = []
fcnt = 0
for rootdir in folders:
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            if os.path.exists(os.path.join(d, "latest_result.pkl")):
                fcnt+=1
                with open(os.path.join(d, "config.json"), "r") as jf:
                    config = json.load(jf)
                    config = flatten_dict(config)
                with open(os.path.join(d, "latest_result.pkl"), "rb") as jf:
                    results = pickle.load(jf)
                if config['layers'] == "100,100,50,20,10" and config["glrt_scale"] == 3 and config['epsilon_decay'] == "sqrt":
                    print(d, results['expected_regret'][-1])
                # rank = config['newrank']
                # algo = config['algo']
                # layers = config['layers']
                # name = f"{algo}_{rank}_{layers}"
                for k in results.keys():
                    config[k] = [results[k]]
                
                result_keys = np.unique(result_keys + list(results.keys())).tolist()
                # else:
                #     assert np.all([el in result_keys for el in results.keys()])
                tdf = pd.DataFrame.from_dict(config)
                if df is None:
                    df = tdf
                else:
                    df = pd.concat((df,tdf))
            else:
                print(f"{d} not finished")



assert df.shape[0] == fcnt
print(f"result_keys:\n {result_keys}")
print(f"df.columns.to_list():\n {df.columns.to_list()}")

In [ ]:
yy = df[(df.layers == "100,100,50,20,10") & (df.glrt_scale == 3) & (df.epsilon_decay == "sqrt")]
print(yy.shape)

plt.figure(figsize=(10,10))
for i in range(yy.shape[0]):
#     print(el)
    plt.plot(yy.iloc[i].expected_regret, label=i)
plt.legend()

In [ ]:
import torch
import torch.nn as nn
# fn = "/private/home/pirotta/lrcb/wheel_0524/nnegreedy_minfeat/1/model_state_dict_n379020.pt"
# fn_a = "/private/home/pirotta/lrcb/wheel_0524/nnegreedy_minfeat/1/algo.pt"

# fn = "/private/home/pirotta/lrcb/wheel_0524/nnegreedy_mse/1/model_state_dict_n379020.pt"
# fn_a = "/private/home/pirotta/lrcb/wheel_0524/nnegreedy_mse/1/algo.pt"

fn = "/private/home/pirotta/lrcb/wheel_0524/nnegreedy_rayleigh/1/model_state_dict_n379020.pt"
fn_a = "/private/home/pirotta/lrcb/wheel_0524/nnegreedy_rayleigh/1/algo.pt"

fn = "/private/home/pirotta/lrcb/wheel_0524/nnegreedy_rayleigh/1/model_state_dict_n768.pt"

model = MLLinearNetwork(7, [(100, nn.ReLU()),(100, nn.ReLU()),(50, nn.ReLU()),(20, nn.ReLU()),(10, nn.ReLU())]).to(torch.float64)
model.load_state_dict(torch.load(fn))
model.eval()


payload = torch.load(fn_a)
feature_matrix = payload["features"]
rewards = payload["rewards"]
nc,na,dim = feature_matrix.shape

all_features = feature_matrix.reshape(-1, dim)
features_tensor = torch.tensor(all_features, dtype=torch.float64)
phi = model.embedding(features_tensor)

new_phi = phi.detach().numpy().reshape(nc, na, model.embedding_dim)
hls_rank = hlsutils.hls_rank(new_phi, rewards, tol=1e-4)
ishls = 1 if hlsutils.is_hls(new_phi, rewards, tol=1e-4) else 0
hls_lambda = hlsutils.hls_lambda(new_phi, rewards)
rank_phi = hlsutils.rank(new_phi, tol=1e-4)

print(f"hls_rank: {hls_rank}")
print(f"hls_lambda: {hls_lambda}")
print(f"rank_phi: {rank_phi}")

from sklearn.manifold import TSNE, SpectralEmbedding, MDS, Isomap
import seaborn as sns

# We want to get TSNE embedding with 2 dimensions
n_components = 2
tsne = TSNE(n_components=n_components)
tsne_result = tsne.fit_transform(phi.detach().numpy())
print(tsne_result.shape)

opt_phi = (rewards.reshape(-1,1) > 1.15).astype(int).squeeze()

tsne_result_df = pd.DataFrame({'tsne_1': tsne_result[:,0], 'tsne_2': tsne_result[:,1], 'label': opt_phi})
fig, ax = plt.subplots(1)
sns.scatterplot(x='tsne_1', y='tsne_2', hue='label', data=tsne_result_df, ax=ax,s=60)
# lim = (tsne_result.min()-5, tsne_result.max()+5)
# ax.set_xlim(lim)
# ax.set_ylim(lim)
ax.set_title("RAY")
ax.set_aspect('equal')
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

In [ ]:
from natsort import natsorted, ns
from celluloid import Camera

files = [
"model_state_dict_n101.pt",
"model_state_dict_n308.pt",
"model_state_dict_n315850.pt",
"model_state_dict_n1107.pt",
"model_state_dict_n33.pt",
"model_state_dict_n11862.pt",
"model_state_dict_n3309.pt",
"model_state_dict_n12.pt",
"model_state_dict_n35423.pt",
"model_state_dict_n122.pt",
"model_state_dict_n370.pt",
"model_state_dict_n126932.pt",
"model_state_dict_n379020.pt",
"model_state_dict_n1329.pt",
"model_state_dict_n3971.pt",
"model_state_dict_n14235.pt",
"model_state_dict_n40.pt",
"model_state_dict_n147.pt",
"model_state_dict_n42508.pt",
"model_state_dict_n15.pt",
"model_state_dict_n444.pt",
"model_state_dict_n4766.pt",
"model_state_dict_n1595.pt",
"model_state_dict_n48.pt",
"model_state_dict_n177.pt",
"model_state_dict_n533.pt",
"model_state_dict_n18.pt",
"model_state_dict_n58.pt",
"model_state_dict_n640.pt",
"model_state_dict_n213.pt",
"model_state_dict_n70.pt",
"model_state_dict_n22.pt",
"model_state_dict_n84.pt",
"model_state_dict_n27.pt",
"model_state_dict_n256.pt",
"model_state_dict_n768.pt",
"model_state_dict_n922.pt",
"model_state_dict_n1914.pt",
"model_state_dict_n2297.pt",
"model_state_dict_n2757.pt",
"model_state_dict_n5720.pt",
"model_state_dict_n6864.pt",
"model_state_dict_n8237.pt",
"model_state_dict_n9885.pt",
"model_state_dict_n17082.pt",
"model_state_dict_n20499.pt",
"model_state_dict_n24599.pt",
"model_state_dict_n29519.pt",
"model_state_dict_n51010.pt",
"model_state_dict_n61212.pt",
"model_state_dict_n73455.pt",
"model_state_dict_n88146.pt",
"model_state_dict_n105776.pt",
"model_state_dict_n152319.pt",
"model_state_dict_n182783.pt",
"model_state_dict_n219340.pt",
"model_state_dict_n263208.pt"
]

fn_a = "/private/home/pirotta/lrcb/wheel_0524/nnegreedy_rayleigh/1/algo.pt"
prefix = "/private/home/pirotta/lrcb/wheel_0524/nnegreedy_rayleigh/1/"
name = "ray"
# prefix = "/private/home/pirotta/lrcb/wheel_0524/nnegreedy_mse/1/"
# name = "MSE"
# prefix = "/private/home/pirotta/lrcb/wheel_0524/nnegreedy_minfeat/1/"
# name = "weak"

payload = torch.load(fn_a)
feature_matrix = payload["features"]
rewards = payload["rewards"]
nc,na,dim = feature_matrix.shape

all_features = feature_matrix.reshape(-1, dim)
features_tensor = torch.tensor(all_features, dtype=torch.float64)

fig = plt.figure(figsize=(5,5))
camera = Camera(fig)

rank_all = []
rank_opt = []

for el in natsorted(files):
    t = el.split("_")[-1]
    t = t.split('.')[0]
    fn = os.path.join(prefix, el)

    model = MLLinearNetwork(7, [(100, nn.ReLU()),(100, nn.ReLU()),(50, nn.ReLU()),(20, nn.ReLU()),(10, nn.ReLU())]).to(torch.float64)
    model.load_state_dict(torch.load(fn))
    model.eval()

    phi = model.embedding(features_tensor)

    new_phi = phi.detach().numpy().reshape(nc, na, model.embedding_dim)
    hls_rank = hlsutils.hls_rank(new_phi, rewards, tol=1e-4)
    ishls = 1 if hlsutils.is_hls(new_phi, rewards, tol=1e-4) else 0
    hls_lambda = hlsutils.hls_lambda(new_phi, rewards)
    rank_phi = hlsutils.rank(new_phi, tol=1e-4)

    rank_all.append(rank_phi)
    rank_opt.append(hls_rank)

    print(f"{t}")
    print(f"hls_rank: {hls_rank}")
    print(f"hls_lambda: {hls_lambda}")
    print(f"rank_phi: {rank_phi}")
    # We want to get TSNE embedding with 2 dimensions
    n_components = 2
    tsne = TSNE(n_components=n_components, learning_rate="auto",init="pca")
    tsne_result = tsne.fit_transform(phi.detach().numpy())
    print(tsne_result.shape)

    opt_phi = (rewards.reshape(-1,1) > 1.15).astype(int).squeeze()
    label_phi = ["optimal actions" if x == 1 else "other actions" for x in opt_phi]

    tsne_result_df = pd.DataFrame({'tsne_1': tsne_result[:,0], 'tsne_2': tsne_result[:,1], 'label': opt_phi})
    # plt.figure(figsize=(5,5))
    dd = tsne_result_df[tsne_result_df.label == 0]
    plt.plot(dd.tsne_1, dd.tsne_2, "bo", markersize=2, label="other actions")
    dd = tsne_result_df[tsne_result_df.label == 1]
    plt.plot(dd.tsne_1, dd.tsne_2, "r*", markersize=8, label="optimal actions")
    # plt.legend()
    plt.xlim((-40,40))
    plt.ylim((-30,30))
    plt.text(0.5, 1.01, f"{name} - {t} - {hls_rank}/{rank_phi}", transform=ax.transAxes)
    camera.snap()
    # fig, ax = plt.subplots(1)
    # sns.scatterplot(x='tsne_1', y='tsne_2', hue='label', data=tsne_result_df, ax=ax, s=60)
    # # lim = (tsne_result.min()-5, tsne_result.max()+5)
    # ax.set_xlim((-40,40))
    # ax.set_ylim((-40,40))
    # ax.set_title(f"RAY - {t} - {hls_rank}/{rank_phi}")
    # ax.set_aspect('equal')
    # ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
animation = camera.animate()
animation.save(f'{name}.mp4')
# from IPython.display import HTML
# HTML(animation.to_html5_video())

rank_all = np.array(rank_all)
rank_opt = np.array(rank_opt)
plt.figure(figsize=(10,5))
plt.plot(rank_all - rank_opt)
plt.title(f"{name}")
plt.show()
plt.savefig(f"{name}_rankdiff.png")